# Predicting Star Ratings


Our objective is to predict a new venue's popularity from information available when the venue opens.  We will do this by machine learning from a data set of venue popularities provided by Yelp.  The data set contains meta data about the venue (where it is located, the type of food served, etc.).  It also contains a star rating. Note that the venues are not limited to restaurants. This tutorial will walk you through one way to build a machine learning algorithm.


## Metrics and scoring

For most questions, you are asked to submit your models `predict` method to the grader. The grader uses a test set to evaluate your model's performance against our reference solution, using the $R^2$ score. It **is** possible to receive a score greater than one, indicating that you've beaten our reference model. We compare our model's score on a test set to your score on the same test set. See how high you can go!

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
sns.set()

import dill
import glob
import collections
from collections import defaultdict
import collections.abc

from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
import ujson as json
import gzip

with gzip.open('yelp_train_academic_dataset_business.json.gz') as f:
    data = [json.loads(line) for line in f]

In [3]:
star_ratings = [row['stars'] for row in data]

X = data
y = star_ratings

# Aims


## Aim 1: city_model

The venues belong to different cities.  You can imagine that the ratings in some cities are probably higher than others.  We wish to build an estimator to make a prediction based on this. 

In [4]:
star_sum = defaultdict(int)
count = defaultdict(int)

In [5]:
for row, stars in zip(data, star_ratings):
    # increment the running sum in star_sum
    # increment the running count in count
    city = row['city']
    star_sum[city] += stars
    count[city] += 1

Now we can calculate the average ratings.  Again, a dictionary makes a good container.

In [6]:
len(star_sum), len(count)

(167, 167)

In [7]:
avg_stars = dict()

for city in star_sum:
    # calculate average star rating and store in avg_stars
    avg_stars[city] = star_sum[city]/count[city]
avg_stars['Phoenix']

3.6702903946388683

There should be 167 different cities:

In [8]:
class CityRegressor(BaseEstimator, RegressorMixin):
    
    def __init__(self):
        self.avg_stars = dict()
    
    def fit(self, X, y):
        # Store the average rating per city in self.avg_stars
        star_sum = defaultdict(int)
        count = defaultdict(int)
        for row, stars in zip(X, y):
            city = row['city']
            star_sum[city] += stars
            count[city] += 1
        for city in star_sum:
            self.avg_stars[city] = star_sum[city]/count[city]
        return self
    
    def predict(self, X):  
        return [self.avg_stars[row['city']] if row['city'] in self.avg_stars else 0 for row in X  ]


In [9]:
city_model = CityRegressor()

city_model.fit(X, y).predict(X[:5])

[3.6702903946388683, 3.75, 3.75, 3.75, 3.75]

In [10]:
city_model.predict([{'city': 'Phoenix'}, {'city': 'Timbuktu'}, {'city': 'Madison'}])

[3.6702903946388683, 0, 3.6457337883959045]

In [11]:
with open('city_model.dill', 'wb') as f:
    dill.dump(city_model, f)

## Aim 2: lat_long_model

You can imagine that a city-based model might not be sufficiently fine-grained. For example, we know that some neighborhoods are trendier than others.  Use the latitude and longitude of a venue as features that help you understand neighborhood dynamics.

In [12]:
class ToDataFrame(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # This transformer doesn't need to learn anything about the data,
        # so it can just return self without any further processing
        return self
    
    def transform(self, X):
        # Return a pandas data frame from X
        return pd.DataFrame(X)

In [14]:
selector = ColumnTransformer([
    ('lat', 'passthrough', ['latitude']),
    ('lon', 'passthrough', ['longitude'])
])

In [16]:
# Training the model
to_data_frame = ToDataFrame()
data_transform = to_data_frame.transform(data)
data_transform = selector.fit_transform(data_transform)
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(data_transform, star_ratings)

# Making predictions
test_data = data[:5]
test_data_transform = to_data_frame.transform(test_data)
test_data_transform = selector.transform(test_data_transform)
knn.predict(test_data_transform)

array([4. , 4.2, 4. , 3.8, 4.2])

In [17]:
pipe_lat_long = Pipeline([
    ('to_data_frame', ToDataFrame()),
    ('column selection', selector),
    ('regressor', knn)
])
pipe_lat_long.fit(X, y).predict(data[:5])

array([4. , 4.2, 4. , 3.8, 4.2])

In [18]:
pipe_lat_long.get_params()

{'memory': None,
 'steps': [('to_data_frame', ToDataFrame()),
  ('column selection',
   ColumnTransformer(transformers=[('lat', 'passthrough', ['latitude']),
                                   ('lon', 'passthrough', ['longitude'])])),
  ('regressor', KNeighborsRegressor())],
 'verbose': False,
 'to_data_frame': ToDataFrame(),
 'column selection': ColumnTransformer(transformers=[('lat', 'passthrough', ['latitude']),
                                 ('lon', 'passthrough', ['longitude'])]),
 'regressor': KNeighborsRegressor(),
 'column selection__n_jobs': None,
 'column selection__remainder': 'drop',
 'column selection__sparse_threshold': 0.3,
 'column selection__transformer_weights': None,
 'column selection__transformers': [('lat', 'passthrough', ['latitude']),
  ('lon', 'passthrough', ['longitude'])],
 'column selection__verbose': False,
 'column selection__lat': 'passthrough',
 'column selection__lon': 'passthrough',
 'regressor__algorithm': 'auto',
 'regressor__leaf_size': 30,
 'regr

In [19]:
param_grid = {'regressor__n_neighbors': np.arange(1, 25)}
gs_est = GridSearchCV(pipe_lat_long, param_grid, cv=3, n_jobs=2, verbose=1)
gs_est.fit(X, y)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('to_data_frame', ToDataFrame()),
                                       ('column selection',
                                        ColumnTransformer(transformers=[('lat',
                                                                         'passthrough',
                                                                         ['latitude']),
                                                                        ('lon',
                                                                         'passthrough',
                                                                         ['longitude'])])),
                                       ('regressor', KNeighborsRegressor())]),
             n_jobs=2,
             param_grid={'regressor__n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])},
             verbose=1)

In [20]:
gs_est.best_params_ #the best_params keeps increasing as the range increases, does this sound right?

{'regressor__n_neighbors': 24}

In [21]:
gs_est.best_score_

-0.06528733598592444

In [22]:
lat_long_model = gs_est.best_estimator_

In [23]:
with open('lat_long_model.dill', 'wb') as f:
    dill.dump(lat_long_model, f)

## Aim 3: category_model

While location is important, we could also try seeing how predictive the
venue's category is. Build an estimator that considers only the `'categories'` field of the data.

In [24]:
class DictEncoder(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # X will be a pandas series. Return a pandas series of dictionaries
        return X.apply(lambda X: dict.fromkeys(X, 1))

In [26]:
vec = Pipeline([
    ('encoder', DictEncoder()),
    ('vectorizer', DictVectorizer())
])

In [27]:
Selector_cat = ColumnTransformer([
    ('vectorized categories', vec, 'categories')
])

In [28]:
category_model = Pipeline([
    ('to_df', ToDataFrame()),
    ('feature_selection', Selector_cat),
    ('regressor', Ridge())
])

In [61]:
category_model.fit(X, y).predict(X[:5])

array([3.33942676, 3.34706925, 3.28311091, 3.22129515, 3.30411137])

In [30]:
with open('category_model.dill', 'wb') as f:
    dill.dump(category_model, f)

## Aim 4: attribute_model

There is even more information in the attributes for each venue.  Let's build an estimator based on these.

Venues attributes may be nested:
```python
{
  'Attire': 'casual',
  'Accepts Credit Cards': True,
  'Ambiance': {'casual': False, 'classy': False}
}
```
We wish to encode them in the same manner as our categories data using the `DictVectorizer`. 

In [32]:
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.abc.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        elif isinstance(v, bool):
            items.append((new_key, int(v)))
        else:
            items.append((new_key, 1))
    return dict(items)

In [33]:
class DictFlat(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # X will be a pandas series. Return a pandas series of dictionaries
        return X.apply(lambda X: flatten(X))

In [34]:
vec = Pipeline([
    ('encoder', DictFlat()),
    ('vectorizer', DictVectorizer())
])

Selector_att = ColumnTransformer([
    ('vectorized attributes', vec, 'attributes')
])

pipe_trans = Pipeline([
    ('to_df', ToDataFrame()),
    ('feature_selection', Selector_att),
])

X_trans = pipe_trans.fit_transform(X)

<37938x78 sparse matrix of type '<class 'numpy.float64'>'
	with 531447 stored elements in Compressed Sparse Row format>

In [35]:
param_grid = {'alpha': np.logspace(-3,3,20)}
gs_ridge = GridSearchCV(Ridge(), param_grid, cv=3, n_jobs=2, verbose=1)
gs_ridge.fit(X_trans, y)
print(gs_ridge.best_params_)
print(gs_ridge.best_score_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
{'alpha': 12.742749857031322}
0.0033902783589166616


In [36]:
gs_ridge.best_estimator_.predict(X_trans[:5])

array([4.052435  , 3.42818792, 3.58667198, 3.27442234, 3.26061997])

In [37]:
rf = RandomForestRegressor(random_state=42)

In [38]:
class EnsembleRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, lin, nonlin):
        self.lin = lin
        self.nonlin = nonlin
        
    def fit(self, X, y):
        self.lin.fit(X, y)
        y_res = y - self.lin.predict(X)
        self.nonlin.fit(X, y_res)
        return self
    
    def predict(self, X):        
        return (self.nonlin.predict(X)+self.lin.predict(X))

In [39]:
regressor = EnsembleRegressor(gs_ridge.best_estimator_, rf)

In [40]:
regressor.fit(X_trans, y)

EnsembleRegressor(lin=Ridge(alpha=12.742749857031322),
                  nonlin=RandomForestRegressor(random_state=42))

In [41]:
regressor.get_params()

{'lin__alpha': 12.742749857031322,
 'lin__copy_X': True,
 'lin__fit_intercept': True,
 'lin__max_iter': None,
 'lin__normalize': False,
 'lin__random_state': None,
 'lin__solver': 'auto',
 'lin__tol': 0.001,
 'lin': Ridge(alpha=12.742749857031322),
 'nonlin__bootstrap': True,
 'nonlin__ccp_alpha': 0.0,
 'nonlin__criterion': 'mse',
 'nonlin__max_depth': None,
 'nonlin__max_features': 'auto',
 'nonlin__max_leaf_nodes': None,
 'nonlin__max_samples': None,
 'nonlin__min_impurity_decrease': 0.0,
 'nonlin__min_impurity_split': None,
 'nonlin__min_samples_leaf': 1,
 'nonlin__min_samples_split': 2,
 'nonlin__min_weight_fraction_leaf': 0.0,
 'nonlin__n_estimators': 100,
 'nonlin__n_jobs': None,
 'nonlin__oob_score': False,
 'nonlin__random_state': 42,
 'nonlin__verbose': 0,
 'nonlin__warm_start': False,
 'nonlin': RandomForestRegressor(random_state=42)}

In [42]:
params_reg = {'nonlin__max_depth': np.arange(1,16),
             'nonlin__min_samples_leaf': np.arange(1,4)}

gs_reg = GridSearchCV(regressor, params_reg, cv=3, n_jobs=2, verbose=1)
gs_reg.fit(X_trans, y)

Fitting 3 folds for each of 45 candidates, totalling 135 fits


GridSearchCV(cv=3,
             estimator=EnsembleRegressor(lin=Ridge(alpha=12.742749857031322),
                                         nonlin=RandomForestRegressor(random_state=42)),
             n_jobs=2,
             param_grid={'nonlin__max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]),
                         'nonlin__min_samples_leaf': array([1, 2, 3])},
             verbose=1)

In [43]:
gs_reg.best_params_

{'nonlin__max_depth': 15, 'nonlin__min_samples_leaf': 3}

In [44]:
gs_reg.best_estimator_.predict(X_trans[:5])

array([3.90168891, 3.4389031 , 3.55879558, 3.80417926, 3.40617893])

In [45]:
attribute_model = Pipeline([
    ('to_df', ToDataFrame()),
    ('feature_selection', Selector_att),
    ('regressor', gs_reg.best_estimator_)
])

In [46]:
attribute_model.fit(X, y).predict(X[:5])

Pipeline(steps=[('to_df', ToDataFrame()),
                ('feature_selection',
                 ColumnTransformer(transformers=[('vectorized attributes',
                                                  Pipeline(steps=[('encoder',
                                                                   DictFlat()),
                                                                  ('vectorizer',
                                                                   DictVectorizer())]),
                                                  'attributes')])),
                ('regressor',
                 EnsembleRegressor(lin=Ridge(alpha=12.742749857031322),
                                   nonlin=RandomForestRegressor(max_depth=15,
                                                                min_samples_leaf=3,
                                                                random_state=42)))])

In [47]:
with open('attribute_model.dill', 'wb') as f:
    dill.dump(attribute_model, f)

## Aim 5: full_model

So far we have only built models based on individual features.  Now we will build an ensemble regressor that averages together the estimates of the four previous regressors.

In [49]:
class ModelTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self, model):
        # What needs to be done here?
        self.model = model
        
    def fit(self, X, y):
        # Fit the stored predictor.
        # Question: what should be returned?
        self.model.fit(X, y)
        return self
    
    def transform(self, X):
        # Use predict on the stored predictor as a "transformation".
        # Be sure to return a 2-D array.
        return np.array(self.model.predict(X)).reshape(-1,1)

Create an instance of `ModelTransformer` for each of the previous four models. Combine these together in a single feature matrix with a
[`FeatureUnion`](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.FeatureUnion.html#sklearn.pipeline.FeatureUnion).

In [50]:
with open('city_model.dill', 'rb') as f:
    city_model = dill.load(f)
city_trans = ModelTransformer(city_model)

In [51]:
with open('lat_long_model.dill', 'rb') as f:
    lat_long_model = dill.load(f)
lat_long_trans = ModelTransformer(lat_long_model)

In [52]:
with open('category_model.dill', 'rb') as f:
    category_model = dill.load(f)
category_trans = ModelTransformer(category_model)

In [53]:
with open('attribute_model.dill', 'rb') as f:
    attribute_model = dill.load(f)  
attribute_trans = ModelTransformer(attribute_model)

In [54]:
union = FeatureUnion([
        ('city', city_trans),
        ('lat_long', lat_long_trans),
        ('categories', category_trans),
        ('attributes', attribute_trans)
])

In [56]:
full_model = Pipeline([
    ('predictors', union),
    ('regressor', Ridge())
])

In [60]:
full_model.fit(X, y).predict(X[:5])

array([3.77054705, 3.36661673, 3.41654682, 3.43346548, 3.39675801])